![](ubc_header.png)

# Climate-Friendly Food Systems (CFFS) Labelling Project

### The University of British Columbia

****

## Part I: Data Preprocessing

## Set up and Import Libraries

In [1]:
#pip install -r requirements.txt

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import os
import csv
from itertools import islice
from decimal import Decimal
import xml.etree.ElementTree as et
from xml.etree.ElementTree import parse
import openpyxl
import pytest

In [3]:
# RUN ONLY ONCE
# os.chdir is used to change the current directory to the specified path
os.chdir("../") # Sets path to the repo folder as it is one level above where this file exists!
path = os.getcwd()
print(path)

/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024


****

## Load Data Files

### Set Data File Path

In [4]:
# Select data file path for the chosen venue and time range where the recipes data stored

# OK 2023
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "OK", "*.oc"))

# Totem 2023
# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "Feast", "*.oc"))

# Gather 2023
filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "Gather", "*.oc"))

# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "BLUEBERRY_FESTIVAL_24"))

# filepath_list = glob.glob(os.path.join(os.getcwd(), "data", "raw", "HARVEST_FESTIVAL_24"))

filepath_list

['/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/raw/Gather/ALL_gather_menu_july30.oc',
 '/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/raw/Gather/missing_gather_items_15_july.oc',
 '/Users/vivaanwadhwa/Documents/GitHub/CFFS_sharon_2024/data/raw/Gather/Gather_missing_items.oc.oc']

### Import Items List

In [5]:
# Read items.xml files in the filepath_list and construct a dataframe
ItemId = []
Description = []
CaseQty = []
CaseUOM = []
PakQty = []
PakUOM = []
InventoryGroup = []


# from the items xml file, findtext of CaseQty, CaseUOM, PakQty, PakUOM, and InventoryGroup
# then append it on the lists above

for filepath in filepath_list:
    path = filepath + '/Items.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for item in xtree.iterfind('Item'):
            ItemId.append(item.attrib['id'])
            Description.append(item.findtext('Description'))
            CaseQty.append(item.findtext('CaseQty'))
            CaseUOM.append(item.findtext('CaseUOM'))
            PakQty.append(item.findtext('PakQty'))
            PakUOM.append(item.findtext('PakUOM'))
            InventoryGroup.append(item.findtext('InventoryGroup'))

            
# Create a dataframe from the lists created above.
        
Items = pd.DataFrame({'ItemId': ItemId, 'Description': Description, 'CaseQty': CaseQty, 
                      'CaseUOM': CaseUOM, 'PakQty': PakQty, 'PakUOM': PakUOM, 'InventoryGroup': InventoryGroup}
                    )

Items.drop_duplicates(inplace=True)

Items.reset_index(drop=True, inplace=True)

In [6]:
# creates a new array with unique ItemIds
all_id_list = Items["ItemId"].unique()

In [7]:
# None of the items are egg yolk liq
Items.loc[Items["Description"] == "Egg Yolk Liq"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [8]:
# Sumplemental option for egg, vegan option
Items.loc[Items["ItemId"] == "I-68700"]

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup


In [9]:
# Gives you the list of breads
breadlist = []


# If the items in the Items list is a "LOAF", "SANDWICH", or "BREAD" then we should add it in the breadlist
# We append it onto the breadlist at position row["ItemId"]
for ind, row in Items.iterrows():
    if ("LOAF" or "SANDWICH" "BREAD") in row["Description"]:
        breadlist.append(row["ItemId"])

breadlist

['I-19634',
 'I-13420',
 'I-1271',
 'I-11664',
 'I-73353',
 'I-24229',
 'I-15032',
 'I-1274',
 'I-12478',
 'I-11842']

In [10]:
Items = Items[~Items["InventoryGroup"].isin(["DISPOSABLES", "SUPPLY"])]

In [11]:
# Based on info below there are 486 rows and 7 columns
Items.shape

(675, 7)

In [12]:
Items.dtypes

ItemId            object
Description       object
CaseQty           object
CaseUOM           object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [13]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Items_List.csv")
Items.to_csv(path, index = False, header = True)

### Import Ingredients List

In [14]:
# Read ingredients.xml files in the filepath_list and construct a dataframe
IngredientId = []
Conversion = []
InvFactor = []
Qty = []
Recipe = []
Uom = []

# Using the Ingredients XML file, we extract attributes containing ingredients, conversion, invFactor, qty, recipe, and uom. 
# Then we append it onto the IngredientId, Coversion, InvFactor, Qty, Recipe, and Uom lists
# Then we create a dataframe using the lists created. 

for filepath in filepath_list:
    path = filepath + '/Ingredients.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Ingredient'):
            IngredientId.append(x.attrib['ingredient'])
            Conversion.append(x.attrib['conversion'])
            InvFactor.append(x.attrib['invFactor'])
            Qty.append(x.attrib['qty'])
            Recipe.append(x.attrib['recipe'])
            Uom.append(x.attrib['uom'])
    
Ingredients = pd.DataFrame({'IngredientId': IngredientId, 'Qty': Qty,'Uom': Uom, 'Conversion': Conversion, 
                      'InvFactor': InvFactor,'Recipe': Recipe}).drop_duplicates()
Ingredients.drop_duplicates(subset=["IngredientId", "Recipe"], inplace=True)

Ingredients.reset_index(drop=True, inplace=True)

In [15]:
Ingredients.loc[Ingredients["IngredientId"] == "I-29389"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
1570,I-29389,2.500,Kg,1.00000000,1.0000,P-50739
2210,I-29389,0.700,Kg,1.00000000,0.1286,P-56431


In [16]:
# This will output the IngredientId on the right side and the number of times the ingredient appears in the Ingredients
# dataframe on the left side. 

# The duplicated() method returns a Series with True and False values that describe which rows in the DataFrame are 
# duplicated and not.

check = Ingredients["IngredientId"].duplicated()

# The line below tells us at which index is the ingredient duplicated. For example I-4598 exists on index 2 of the dataframe and
# then again on index 8. So duplicate is first true on index 8, which is why it is printed below. It also appears in a below 
# index so it is printed again with that index number. 
Ingredients["IngredientId"][check]

5       P-18746
21      I-12176
26       I-3642
28       I-3387
36       I-5983
         ...   
5491     P-9714
5492    I-22493
5493    P-26514
5494    P-44913
5495    P-50725
Name: IngredientId, Length: 3908, dtype: object

In [17]:
# We can see from the printed items above that I-64877 does not appear because it does not get duplicated
Ingredients.loc[Ingredients["IngredientId"] == "I-64877"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [18]:
Ingredients

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe
0,P-9857,5.000,Kg,1.00000000,1.1111,P-10112
1,P-48933,1.000,Kg,1.00000000,1.0000,P-10113
2,I-4626,16.000,CT,1.00000000,2.6667,P-10115
3,P-9755,1.000,lb,1.00000000,1.0000,P-10119
4,P-18746,1.000,Kg,1.00000000,1.0000,P-10241
...,...,...,...,...,...,...
5492,I-22493,5.000,g,0.00100000,1.0000,R-73668
5493,P-26514,2.000,g,1.00000000,1.0000,R-73668
5494,P-44913,90.000,g,0.00100000,1.0000,R-73668
5495,P-50725,1.000,ea,1.00000000,1.0000,R-73668


In [19]:
# Below we are checking if we can extract an entire recipe. So we can do .loc with the particular recipe id and then 
# print all the data points (ingredientId, Qty, Uom, Conversion, InvFactor) for that recipe. 
Ingredients.loc[Ingredients["Recipe"] == "R-68698"]

,IngredientId,Qty,Uom,Conversion,InvFactor,Recipe


In [20]:
Items

,ItemId,Description,CaseQty,CaseUOM,PakQty,PakUOM,InventoryGroup
0,I-7631,5 SPICE POWDER,1.000,ea,1.000,lb,SPICES
1,I-43851,AJWAIN 5LB,5.000,lb,454.000,g,SPICES
2,I-3617,ALLSPICE GROUND SHAKER,8.000,each,454.000,g,SPICES
3,I-1852,ALMOND BEV - ORIGINAL,24.000,CT,1.000,CT,BEVERAGE
4,I-44059,ALMOND BUTTER*,10.000,Kg,1.000,Kg,FOOD - GROCERY
...,...,...,...,...,...,...,...
670,I-3642,PEPPER BLK*COARSE GRD,2.100,Kg,1000.000,g,SPICES
671,I-6026,SALT KOSHER*,9.000,pak,1360.780,g,SPICES
672,I-12176,SALT SEA*COARSE KOSHER,1.000,CT,2.270,Kg,SPICES
673,I-4904,STRAWBERRIES (CLAMSHELL) US,8.000,lb,1.000,lb,PRODUCE


In [21]:
Ingredients.shape

(5497, 6)

In [22]:
Ingredients.dtypes

IngredientId    object
Qty             object
Uom             object
Conversion      object
InvFactor       object
Recipe          object
dtype: object

In [23]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Ingredients_List.csv")
Ingredients.to_csv(path, index = False, header = True)

### Import Preps List

In [24]:
# Read preps.xml files in the filepath_list and construct a dataframe
PrepId = []
Description = []
PakQty = []
PakUOM = []
InventoryGroup = []


# Here we do the same thing for the Preps XML file where we find the columns using attrib function and then append it onto
# the dataframe called Preps. 

for filepath in filepath_list:
    path = filepath + '/Preps.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prep'):
            PrepId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            PakQty.append(x.findtext('PakQty'))
            PakUOM.append(x.findtext('PakUOM'))
            InventoryGroup.append(x.findtext('InventoryGroup'))
    
Preps = pd.DataFrame({'PrepId': PrepId, 'Description': Description,
                  'PakQty': PakQty, 'PakUOM':PakUOM, 'InventoryGroup': InventoryGroup}).drop_duplicates()
preps_columns = Preps.columns
Preps.drop_duplicates(subset=["PrepId"], inplace=True)

Preps.reset_index(drop=True, inplace=True)

In [25]:
Preps

,PrepId,Description,PakQty,PakUOM,InventoryGroup
0,P-68207,24|BATCH|Poblano Pork Stew*,100.000,PTN,PREP
1,P-61322,AIOLI|Pesto,4.000,L,ZDONT USE OK - PREP
2,P-46066,Almond Milk Yeild,900.000,ml,
3,P-8814,Aoli- RoastedGarlic Thyme,1.100,L,
4,P-49605,APPLE|Cubed,75.000,g,
...,...,...,...,...,...
898,P-54352,Garnish|Avocado Spinach Pasta,20.000,g,
899,P-73667,PREP|Chili Crunch|SL,4.500,L,FEAST
900,P-73472,PREP|SALAD|Tomato CucMint|MED,6.000,Kg,
901,P-70825,PREP|Sliced Strawberry*,1.300,Kg,


In [26]:
# This tells us that there are not any duplicates in the Preps dataframe. There is a different process|ingredient pair for 
# all the recipes. 
check = Preps["PrepId"].duplicated().any()
print(check)

False


In [27]:
# There are 546 rows and 5 columns
Preps.shape

(903, 5)

In [28]:
# Here we see there is only one place P-50739 is used in the Preps
Preps.loc[Preps["PrepId"] == "P-50739"]

,PrepId,Description,PakQty,PakUOM,InventoryGroup
70,P-50739,COMPOTE|Apple Cinnamon|AAD,2.500,L,PREP


In [29]:
Preps.dtypes

PrepId            object
Description       object
PakQty            object
PakUOM            object
InventoryGroup    object
dtype: object

In [30]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Preps_List.csv")
Preps.to_csv(path, index = False, header = True)

### Import Products List

In [31]:
# Read products.xml files in the filepath_list and construct a dataframe
ProdId = []
Description = []
SalesGroup = []

# From the XML file for products append the id into ProdId, description into Description, and SalesGroup into SalesGroup list.
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe

for filepath in filepath_list:
    path = filepath + '/Products.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Prod'):
            ProdId.append(x.attrib['id'])
            Description.append(x.findtext('Description'))
            SalesGroup.append(x.findtext('SalesGroup'))
        
Products = pd.DataFrame({'ProdId': ProdId, 'Description': Description, 'SalesGroup': SalesGroup})
Products.drop_duplicates(inplace=True)

Products.reset_index(drop=True, inplace=True)

In [32]:
Products

,ProdId,Description,SalesGroup
0,R-73669,DEL|smoothie| mango pineapple,GV - DELISH
1,R-56613,DEL|Smoothie|Almonds forever,GV - DELISH
2,R-44306,DEL|Smoothie|Banana Matcha,GV - DELISH
3,R-43697,DEL|Smoothie|Blueberry Boost,GV - DELISH
4,R-46074,DEL|Smoothie|Bna Almnd Buta,GV - DELISH
...,...,...,...
397,R-72778,SL|BBQ Porkchop|AAD,GV SIMPLY LOCAL
398,R-72819,SL|Lemon Pepper Chichen|AAD,GV SIMPLY LOCAL
399,R-70824,DEL|Strawberry Salad|AAD*,GV - DELISH
400,R-46347,FF|Pasta|Avocado Spin old,GV FRESH FARE DAY


In [33]:
# Here we can see that there is only one R-56966 in the products 
Products.loc[Products["ProdId"] == "R-56966"]

,ProdId,Description,SalesGroup


In [34]:
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [35]:
# Here we can see that there is only one R-68698 in the products 
Products.loc[Products["ProdId"] == "R-68698"]

,ProdId,Description,SalesGroup


In [36]:
Products.shape

(402, 3)

In [37]:
Products.dtypes

ProdId         object
Description    object
SalesGroup     object
dtype: object

In [38]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Products_List.csv")
Products.to_csv(path, index = False, header = True)

### Import Conversions List

In [39]:
# Read conventions.xml files in the filepath_list and construct a dataframe
ConversionId = []
Multiplier = []
ConvertFromQty = []
ConvertFromUom = []
ConvertToQty = []
ConvertToUom = []

# From the XML file for Conversions append the id into ConversionId, multiplier into Multiplier, ConvertFrom->qty into 
# ConvertFromQty,ConvertFrom->uom into ConvertFromUom, ConvertTo->qty into ConvertToQty and and ConvertTo->uom into the
# CovertToUom list. 
# Make a dataframe out of the 3 lists
# Then also drop the duplicates in the Products dataframe


for filepath in filepath_list:
    path = filepath + '/Conversions.xml'
    if os.path.isfile(path):
        xtree = et.parse(path)
        xroot = xtree.getroot()
        for x in xtree.iterfind('Conversion'):
            ConversionId.append(x.attrib['id'])
            Multiplier.append(x.attrib['multiplier'])
            ConvertFromQty.append(x.find('ConvertFrom').attrib['qty'])
            ConvertFromUom.append(x.find('ConvertFrom').attrib['uom'])
            ConvertToQty.append(x.find('ConvertTo').attrib['qty'])
            ConvertToUom.append(x.find('ConvertTo').attrib['uom'])
    
    
Conversions = pd.DataFrame({'ConversionId': ConversionId, 'Multiplier': Multiplier, 'ConvertFromQty': ConvertFromQty,
                           'ConvertFromUom': ConvertFromUom, 'ConvertToQty': ConvertToQty, 'ConvertToUom': ConvertToUom}
                          ).drop_duplicates()

Conversions.reset_index(drop=True, inplace=True)

In [40]:
# Here we can see for example that to convert 1.14 L to 1 L the multiplier is 0.877 since 1/1.14 = 0.877
Conversions

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
0,,1.00000000,1.0000,XXX,1.0000,L
1,,0.87719298,1.0000,1.14L,1.1400,L
2,,0.66666667,1.0000,1.5L,1.5000,L
3,,0.57142857,1.0000,1.75 L,1.7500,L
4,,0.50000000,1.0000,2L,2.0000,L
...,...,...,...,...,...,...
329,I-43958,0.02000000,1.0000,each,50.0000,g
330,I-45114,0.03333333,0.5000,fl oz,15.0000,g
331,I-47525,0.00408163,1.0000,cup,245.0000,g
332,I-56382,0.00100000,1.0000,L,1000.0000,g


In [41]:
# Here we can check that there are no ingredients listed since 
Conversions.loc[Conversions["ConversionId"] == "I-4582"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom
243,I-4582,0.10000000,1.0000,CT,10.0000,g


In [42]:
all_id_list = Items["ItemId"].unique()
all_conv_list = Conversions["ConversionId"].unique()

print("All unique IDs list\n")
print(all_id_list)
print("\n")
print("All unique Conversions list\n")
print(all_conv_list)

All unique IDs list

['I-7631' 'I-43851' 'I-3617' 'I-1852' 'I-44059' 'I-43854' 'I-43652'
 'I-2852' 'I-2853' 'I-6924' 'I-11181' 'I-4255' 'I-29389' 'I-4458'
 'I-13253' 'I-18918' 'I-4971' 'I-4465' 'I-4472' 'I-4973' 'I-65741'
 'I-27410' 'I-63697' 'I-1252' 'I-1207' 'I-18089' 'I-4495' 'I-4496'
 'I-4491' 'I-3119' 'I-22443' 'I-4507' 'I-3130' 'I-3133' 'I-3141' 'I-3310'
 'I-38906' 'I-53706' 'I-3619' 'I-2567' 'I-2866' 'I-23336' 'I-3143'
 'I-4518' 'I-3144' 'I-7935' 'I-3145' 'I-3146' 'I-10616' 'I-3743' 'I-3149'
 'I-34065' 'I-3838' 'I-37002' 'I-3846' 'I-7064' 'I-17670' 'I-11661'
 'I-72216' 'I-3867' 'I-4521' 'I-13050' 'I-32246' 'I-7185' 'I-4531'
 'I-2574' 'I-43648' 'I-4538' 'I-41921' 'I-73416' 'I-73415' 'I-4546'
 'I-31126' 'I-63874' 'I-67659' 'I-72414' 'I-63034' 'I-6020' 'I-8060'
 'I-4550' 'I-4558' 'I-4557' 'I-4556' 'I-13635' 'I-3159' 'I-3586' 'I-11213'
 'I-3620' 'I-4582' 'I-10871' 'I-4589' 'I-4572' 'I-4590' 'I-4574' 'I-11048'
 'I-4593' 'I-3621' 'I-4598' 'I-4599' 'I-34504' 'I-72465' 'I-29346'
 'I-795

In [43]:
# Here we have the number of items in the list that are a part of the "all_id_list" but not part of the "all_conv_list"
missing_conv_id = []

for item in all_id_list:
    if item not in all_conv_list:
        missing_conv_id.append(item)
        
missing_conv_id
print(len(missing_conv_id))

494


In [44]:
Conversions.shape

(334, 6)

In [45]:
Conversions.loc[Conversions["ConversionId"] == "I-29389"]

,ConversionId,Multiplier,ConvertFromQty,ConvertFromUom,ConvertToQty,ConvertToUom


In [46]:
Conversions.dtypes

ConversionId      object
Multiplier        object
ConvertFromQty    object
ConvertFromUom    object
ConvertToQty      object
ConvertToUom      object
dtype: object

In [47]:
# Save the dataframe to csv
path = os.path.join(os.getcwd(), "data", "preprocessed", "Conversions_List.csv")
Conversions.to_csv(path, index = False, header = True)

***
## Data Summary

In [48]:
# Summary of raw data imported for evaluation
# Here we have a summary of the number of items, preps, ingredients, products, conversions

datasum = pd.DataFrame([Items.shape, Preps.shape, Ingredients.shape, Products.shape, Conversions.shape],
                       columns = ['count', 'columns'], 
                       index = ['Items', 'Preps', 'Ingredients', 'Products', 'Conversions'])
datasum

,count,columns
Items,675,7
Preps,903,5
Ingredients,5497,6
Products,402,3
Conversions,334,6
